In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import cv2
import random
from sklearn.cross_validation import train_test_split
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, SGD
from keras.utils import np_utils
from keras.models import load_model
from keras.models import Model
from keras import callbacks
import numpy as np

images=np.load("images.npy")
labels=np.load("labels.npy")

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)
base_model = VGG16(weights='imagenet', include_top=False,input_shape=(96, 96, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

       
sgd = Adam(lr=0.0003) #采用SGD+momentum的优化器进行训练，首先生成一个优化器对象  
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])   #完成实际的模型配置工作
print("train.......")
tbCallbacks = callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)
model.fit(x_train, y_train,
          batch_size=32,epochs = 5,verbose=1,
          validation_data=(x_test, y_test), callbacks=[tbCallbacks])
scroe, accuracy = model.evaluate(x_test, y_test, batch_size=100)
print('scroe:', scroe, 'accuracy:', accuracy) 
model.save("face_96.h5")


Using TensorFlow backend.
/Users/yangbinfeng/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


train.......
Train on 1600 samples, validate on 400 samples
Epoch 1/5
1600/1600 [==============================] - 305s 190ms/step - loss: 0.2431 - acc: 0.9769 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 2/5
1600/1600 [==============================] - 333s 208ms/step - loss: 0.0277 - acc: 0.9981 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 3/5
1600/1600 [==============================] - 344s 215ms/step - loss: 0.0139 - acc: 0.9981 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 4/5
1600/1600 [==============================] - 321s 200ms/step - loss: 9.1960e-04 - acc: 0.9994 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 5/5
400/400 [==============================] - 65s 161ms/step
scroe: 1.19209289551e-07 accuracy: 1.0
